In [1]:
from src import twitterScrapper

### Example of usage

In [ ]:
session = twitterScrapper("Credentials/twitter.json")   # Fill up your own credentials before running this

# NOTE: Please zoom out the browser to 25% so that more posts are loaded per scroll, you can do this by pressing Ctrl + '-' multiple times

In [ ]:
# Example 1 searching for posts 
FILTERS = {
    # Basic filters
    "all_these_words": "",           # Example: what’s happening · contains both “what’s” and “happening”
    "this_exact_phrase": "",         # Example: what’s happening · contains the exact phrase “what’s happening”
    "any_of_these_words": "'",        # Example: what’s happening · contains either “what’s” or “happening”
    "none_of_these_words": "",       # Example: what’s happening · does not contain the words “what’s” or “happening”
    "these_hashtags": "",            # Example: #whatshappening · contains the hashtag #whatshappening
    
    # Account filters
    "from_accounts": "jokowi",    # Example: from:Twitter · Tweets sent from the account Twitter
    "to_accounts": "",            # Example: to:Twitter · Tweets sent in reply to the account Twitter
    "mentioning_accounts": "",    # Example: @Twitter · Tweets that mention the account Twitter

    # Additional filters
    "Minimum_replies": "",         # Example: min_replies:100 · Tweets with at least 100 replies
    "Minimum_likes": "",           # Example: min_faves:100 · Tweets with at least 100 likes
    "Minimum_retweets": "",        # Example: min_retweets:100 · Tweets with at least 100 retweets
    "links": True,                 # Example: filter:links · Include posts with links | If disabled, only posts without links
    "replies": True,               # Example: filter:replies · Include replies and original posts | If disabled, only original posts
}

session.start(
    startDate="",                       # Latest date for scrapping, empty means current date
    endDate="",                         # Earliest date for scrapping, empty means 2006-01-01
    filters=FILTERS,
    scraping_Params={
        "wait_short": 10,               # How long to wait for post to load in seconds after scrollin, etc
        "wait_long": 30,                # How long to wait for page to load in seconds for longer loads
        "detection_wait": 900,          # Wait time in seconds when scraping detection is encountered
        "max_empty_pages": 2            # Maximum number of consecutive empty pages before stopping scrapping
    },
    saveFormat="both",                  # Save both CSV and JSON
    autoSave=True,                      # Enable autosave so `autoSaveInterval` works
    autoSaveInterval=100,               # Save every 100 posts scraped
    continue_if_timeout=True,           # Continue scrapping if scraping detection is encountered
    processDir="jokowi_twitterACC",     # process directory on Process/
    resume_from_savepoint=True          # Resume from latest savepoint if available
)

In [ ]:
# Example 2 searching for posts that have the exact words of either "Makan Bergizi Gratis" or "MBG" in it without any grok response from 2025-12-31 to this day

FILTERS = {
    # Basic filters
    "all_these_words": "",         
    "this_exact_phrase": "",         
    "any_of_these_words": "\'Makan Bergizi Gratis\' \'MBG\'",
    "none_of_these_words": "grok",
    "these_hashtags": "",
    
    # Account filters
    "from_accounts": "",
    "to_accounts": "",
    "mentioning_accounts": "",

    # Additional filters
    "Minimum_replies": "",
    "Minimum_likes": "",
    "Minimum_retweets": "",
    "links": True,
    "replies": True,
}

session.start(
    endDate="2025-12-31",
    filters=FILTERS,
    saveFormat="both",
    autoSave=True,
    autoSaveInterval=100,
    continue_if_timeout=True,   
    processDir="MBG",
    resume_from_savepoint=True
)